In [1]:
import pygame
from random import randint
from math import fabs
from pygame.sprite import Sprite
from pygame.locals import *
#Для работы программы потребуется библиотека pygame. Для ее установки должно быть достаточно pip install pygame
#В некоторых случаях может потребоваться скомпилировать библиотеку вручную
#Всю информацию можно получить тут: https://www.pygame.org
#Класс Object отвечает за объекты, которые непосредственно отображаются на экран
class Object(Sprite):
    def __init__(self, screen, img_filename, x, y, CELL_SIZE, color):
        Sprite.__init__(self)
        self.x = int(x * CELL_SIZE + 0.5 * CELL_SIZE)
        self.y = int(y * CELL_SIZE + 0.5 * CELL_SIZE)
        self.size = CELL_SIZE
        self.color = color
        self.original_image = pygame.image.load(img_filename)
        self.image = pygame.transform.scale(self.original_image,(int(CELL_SIZE), int(CELL_SIZE)))
        self.screen = screen
    def new_x_y(self,x,y):
        self.x = int(x * self.size + 0.5 * self.size)
        self.y = int(y * self.size + 0.5 * self.size)
    def update(self,screen):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), int(self.size/2))
        self.rect = self.image.get_rect(center=(self.x, self.y))
        screen.blit(self.image, self.rect)
#Основная информация об объектах хранится в виде числовой таблицы, с которой работает вся программа
#Объекты анимации создаются лишь в момент отрисовки
def show(screen, Animation_List):
    for line in Animation_List:
        for obj in line:
            if obj!=0:
                #Костыль, иногда координаты не обновляются :(
                obj.new_x_y(Animation_List.index(line),line.index(obj))
                obj.update(screen)
#Функция отрисовки сетки
def draw_grid(screen, size, CELL_SIZE):
    for i in range(size + 1):
        pygame.draw.line(screen, (0,0,0), [i * CELL_SIZE, 0], [i * CELL_SIZE, size * CELL_SIZE], 1)
    for i in range(size + 1):
        pygame.draw.line(screen, (0,0,0), [0, i * CELL_SIZE], [size * CELL_SIZE, i * CELL_SIZE], 1)
#Функция отображения основной информации о кол-ве итераций и кол-ве несчастных ячеек
def show_info(screen, country, size, k ,SCREEN_HEIGHT):
    font = pygame.font.Font(None, 45)
    K_INFO = 'K:' + str(k)
    Un_INFO = 'Unhappy:' + str(round(percentage_of_unhappy(size, country)[1],3)) + '%'
    draw_text1 = font.render(K_INFO, 1, (0,0,0))
    draw_text2 = font.render(Un_INFO, 1, (0,0,0))
    screen.blit(draw_text1, (50, SCREEN_HEIGHT-70))
    screen.blit(draw_text2, (50, SCREEN_HEIGHT-30))
#Функция генерации исходной числовой таблицы. В таблице содержаться числа, отвечающие за "расу" в этой клетке
#если клетка "несчастна", то значение числа не меняется, но оно становится отрицательным
def create_random_country(screen, size, proportion, race_color, CELL_SIZE):
    country = [[0 for i in range(size)] for j in range(size)]
    animation_list = [[0 for i in range(size)] for j in range(size)]
    populated_areas = []
    free_areas = [(i, j) for i in range(size) for j in range(size)]
    for race_id in range(len(proportion[1:])+1):
        race_size = proportion[race_id]
        for i in range(int((size ** 2) * race_size)):
            new_populated_cell_id = randint(0, len(free_areas)-1)
            populated_areas.append(tuple(list(free_areas[new_populated_cell_id]) + [race_id]))
            del free_areas[new_populated_cell_id]
    for habitant in populated_areas:
        if country[habitant[0]][habitant[1]] != habitant[2]:
            country[habitant[0]][habitant[1]] = habitant[2]
            if habitant[2]!=0:
                animation_list[habitant[0]][habitant[1]] = Object(screen,'mask.png',
                                                          habitant[0], habitant[1], CELL_SIZE,
                                                          race_color[int(fabs(habitant[2]))-1])
    return country, animation_list
#Функция получения всех соседних клеток. ВАЖНО: сама клетка тоже попадет в результат
def find_neighbors(country, x, y):
    #Обработка исключений на углах
    if x + y == 0:
        neighbors = country[0][:2] + country[1][:2]
    elif x == 0 and y == len(country)-1:
        neighbors = country[-2][:2] + country[-1][:2]
    elif x == len(country[y])-1 and y == 0:
        neighbors = country[0][-2:] + country[1][-2:]
    elif x == len(country[y])-1 and y == len(country)-1:
        neighbors = country[-2][-2:] + country[-1][-2:]
    #Обработка исключений на границах
    elif x == 0:
        neighbors = country[y-1][:2] + country[y][:2] + country[y + 1][:2]
    elif x == len(country[y])-1:
        neighbors = country[y-1][-2:] + country[y][-2:] + country[y + 1][-2:]
    elif y == 0:
        neighbors = country[0][x-1:x + 2] + country[1][x - 1:x + 2]
    elif y == len(country)-1:
        neighbors = country[-2][x-1:x + 2] + country[-1][x-1:x + 2]
    else:
        neighbors = country[y-1][x-1:x + 2] + country[y][x-1:x + 2] + country[y + 1][x - 1:x + 2]
    return neighbors
#Функция обновляет состояния всех ячеек. 
#Условие строгого неравенства т.к. сама клетка тоже попадает в массив соседей
#Проще так, чем усложнять конструкцию со срезами
def happy_or_unhappy(country):
    for y in range(len(country)):
        for x in range(len(country[y])):
            neighbors = find_neighbors(country, x, y)
            neighbors_of_the_same_color = neighbors.count(country[y][x])+neighbors.count(-country[y][x])
            if neighbors_of_the_same_color > 2:
                country[y][x] = int(fabs(country[y][x]))
            else:
                country[y][x] = int(-fabs(country[y][x]))
#Получение случайной "несчастной" клетки
def find_random_unhappy(country):
    unhappy_list = []
    for y in range(len(country)):
        for x in range(len(country[y])):
            if country[y][x] < 0:
                unhappy_list.append(tuple([x, y]))
    if len(unhappy_list) != 0:
        return unhappy_list[randint(0,len(unhappy_list)-1)]
    else:
        return (None, None)
#Получение случайной свободной клетки
def find_random_free_cell(country):
    free_list = []
    for y in range(len(country)):
        for x in range(len(country[y])):
            if country[y][x] == 0:
                free_list.append(tuple([x,y]))
    return free_list[randint(0, len(free_list)-1)]
#Перемещение "несчастной" клетки по своодному адресу
def sad_move(country, animation):
    unhappy_x, unhappy_y = find_random_unhappy(country)
    free_x, free_y = find_random_free_cell(country)
    if unhappy_x != None:
        country[free_y][free_x], country[unhappy_y][unhappy_x] = country[unhappy_y][unhappy_x], country[free_y][free_x]
        animation[unhappy_y][unhappy_x].new_x_y(free_x, free_y)
        animation[free_y][free_x], animation[unhappy_y][unhappy_x] = animation[unhappy_y][unhappy_x], animation[free_y][free_x]
#Процент "несчастных" клеток
def percentage_of_unhappy(size, country):
    All = [j for i in country for j in i if j != 0]
    unhappys = [i for i in All if i < 0]
    return (len(unhappys), len(unhappys) / len(All) * 100)
#Основная функция с инциализацией и циклом
def run(size, proportion, max_fps, race_colors, step):
    #нормировка пропорции
    proportion = [i/sum(proportion) for i in proportion]
    pygame.init()
    #screen = pygame.display.set_mode((0,0),pygame.FULLSCREEN)
    SCREEN_WIDTH, SCREEN_HEIGHT = [pygame.display.Info().current_h-75]*2
    BG_COLOR = 200, 200, 200
    CELL_SIZE = min(SCREEN_WIDTH, SCREEN_HEIGHT-75) / size
    screen = pygame.display.set_mode((SCREEN_WIDTH-75, SCREEN_HEIGHT),0,32)
    clock = pygame.time.Clock()
    country, animation_list = create_random_country(screen, size, proportion, race_colors, CELL_SIZE)
    happy_or_unhappy(country)
    Mainloop = True
    k=0
    while Mainloop:
        # Limit frame speed to FPS
        time_passed = clock.tick(max_fps)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                Mainloop = False
        if percentage_of_unhappy(size, country)[0] != 0 or (percentage_of_unhappy(size, country)[0] == 0 and k % step != 0):
            k+=1
            #Обновление объектов
            happy_or_unhappy(country)
            sad_move(country,animation_list)
            #Отображение объектов
            if (k % step == 0):
                screen.fill(BG_COLOR)
                show(screen, animation_list)
                draw_grid(screen, size, CELL_SIZE)
                show_info(screen, country, size, k, SCREEN_HEIGHT)
        pygame.display.update()
    pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Тут будут параметры запуска модели
#Размер поля
SIZE = 30
#Пропорции. В данный массив можно указывать соотношения между рассами. Первое число - кол-во свободных клеток
#Можно вводить любые пропорции, все само отнормируется
PROPORTION = [500,30,30,30,30,30,30]
#Цвета задаются случайно, в зависимости от колличества рас.
RACE_COLORS = [tuple(randint(0,255)for c in range(3))for i in range(len(PROPORTION)-1)]
#Исходные данные по умолчанию:
'''PROPORTION = [10,45,45]
RACE_COLORS = [(255,0,0),(0,0,255)]'''
#Максимальный FPS
MAX_FPS = 300
#Шаг отрисовки. Можно произодить отрисовку каждые n шагов (рекомендуется для размеров больше 100)
STEP = 1
run(SIZE, PROPORTION, MAX_FPS, RACE_COLORS, STEP)